# Same as the scmodel.py

In [19]:
#!/usr/bin/env python
# coding: utf-8

import argparse
import logging
import os
import sys
import time
from decimal import Decimal
import glob

import numpy as np
import pandas as pd
import scanpy as sc
import torch
from captum.attr import IntegratedGradients
from numpy.lib.function_base import gradient
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.metrics import (auc, average_precision_score,
                             classification_report, mean_squared_error,
                             precision_recall_curve, r2_score, roc_auc_score)
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import pearsonr
import DaNN.mmd as mmd
import scanpypip.preprocessing as pp
import trainers as t
import utils as ut
from models import (AEBase, CVAEBase, DaNN, Predictor, PretrainedPredictor,
                    PretrainedVAEPredictor, TargetModel, VAEBase)
from scanpypip.utils import get_de_dataframe
from trajectory import trajectory
from sklearn.feature_selection import SelectKBest,SelectFdr
from sklearn.feature_selection import chi2


DATA_MAP={
"GSE117872":"data/GSE117872/GSE117872_good_Data_TPM.txt",
"GSE117309":'data/GSE117309/filtered_gene_bc_matrices_HBCx-22/hg19/',
"GSE117309_TAMR":'data/GSE117309/filtered_gene_bc_matrices_HBCx22-TAMR/hg19/',
"GSE121107":'data/GSE121107/GSM3426289_untreated_out_gene_exon_tagged.dge.txt',
"GSE121107_1H":'data/GSE121107/GSM3426290_entinostat_1hr_out_gene_exon_tagged.dge.txt',
"GSE121107_6H":'data/GSE121107/GSM3426291_entinostat_6hr_out_gene_exon_tagged.dge.txt',
"GSE111014":'data/GSE111014/',
"GSE110894":"data/GSE110894/GSE110894.csv",
"GSE122843":"data/GSE122843/GSE122843.txt",
"GSE112274":"data/GSE112274/GSE112274_cell_gene_FPKM.csv",
"GSE116237":"data/GSE116237/GSE116237_bulkRNAseq_expressionMatrix.txt",
"GSE108383":"data/GSE108383/GSE108383_Melanoma_fluidigm.txt",
"GSE140440":"data/GSE140440/GSE140440.csv",
"GSE129730":"data/GSE129730/GSE129730.h5ad",
"GSE149383":"data/GSE149383/erl_total_data_2K.csv",
"GSE110894_small":"data/GSE110894/GSE110894_small.h5ad",
"MIX-Seq":"data/10298696"

}

# Path of the .pkl files in sc_pre

In [20]:
files = glob.glob("F://ws//pyws//scDEAL//saved//bulk_pre//1214*")

In [21]:
files

['F://ws//pyws//scDEAL//saved//bulk_pre\\1214data_GSE110894_drug_I.BET.762_bottle_128_edim_512,256_pdim_256,128_model_DAE_dropout_0.1_gene_T_lr_0.01_mod_new_sam_downsampling',
 'F://ws//pyws//scDEAL//saved//bulk_pre\\1214data_GSE112274_drug_GEFITINIB_bottle_64_edim_512,256_pdim_256,128_model_DAE_dropout_0.1_gene_T_lr_0.1_mod_new_sam_no',
 'F://ws//pyws//scDEAL//saved//bulk_pre\\1214data_GSE117872HN120_drug_CISPLATIN_bottle_32_edim_256,128_pdim_128,64_model_DAE_dropout_0.3_gene_T_lr_0.5_mod_new_sam_SMOTE',
 'F://ws//pyws//scDEAL//saved//bulk_pre\\1214data_GSE117872HN137_drug_CISPLATIN_bottle_64_edim_256,128_pdim_128,64_model_DAE_dropout_0.3_gene_T_lr_0.1_mod_new_sam_upsampling',
 'F://ws//pyws//scDEAL//saved//bulk_pre\\1214data_GSE140440_drug_DOCETAXEL_bottle_512_edim_256,128_pdim_256,128_model_DAE_dropout_0.3_gene_T_lr_0.01_mod_new_sam_no',
 'F://ws//pyws//scDEAL//saved//bulk_pre\\1214data_GSE149383_drug_ERLOTINIB_bottle_512_edim_512,256_pdim_256,128_model_DAE_dropout_0.1_gene_T_lr_0.1

In [22]:
SELECTED_FILES = 3

# Load arguments

In [23]:
parser = argparse.ArgumentParser()
# data 
parser.add_argument('--data', type=str, default='data/ALL_expression.csv',help='Path of the bulk RNA-Seq expression profile')
parser.add_argument('--label', type=str, default='data/TOTAL_label_binary_wf.csv',help='Path of the processed bulk RNA-Seq drug screening annotation')
parser.add_argument('--result', type=str, default='saved/results/result_',help='Path of the training result report files')
parser.add_argument('--drug', type=str, default='I.BET.762',help='Name of the selected drug, should be a column name in the input file of --label')
parser.add_argument('--missing_value', type=int, default=1,help='The value filled in the missing entry in the drug screening annotation, default: 1')
parser.add_argument('--test_size', type=float, default=0.2,help='Size of the test set for the bulk model traning, default: 0.2')
parser.add_argument('--valid_size', type=float, default=0.2,help='Size of the validation set for the bulk model traning, default: 0.2')
parser.add_argument('--var_genes_disp', type=float, default=None,help='Dispersion of highly variable genes selection when pre-processing the data. \
                     If None, all genes will be selected .default: None')
parser.add_argument('--sampling', type=str, default=None,help='Samping method of training data for the bulk model traning. \
                    Can be upsampling, downsampling, or SMOTE. default: None')
parser.add_argument('--PCA_dim', type=int, default=0,help='Number of components of PCA  reduction before training. If 0, no PCA will be performed. Default: 0')

# trainv
parser.add_argument('--bulk_encoder','-e', type=str, default='saved/bulk_encoder/',help='Path of the pre-trained encoder in the bulk level')
parser.add_argument('--pretrain', type=int, default=1,help='Whether to perform pre-training of the encoder. 0: do not pretraing, 1: pretrain. Default: 0')
parser.add_argument('--lr', type=float, default=1e-2,help='Learning rate of model training. Default: 1e-2')
parser.add_argument('--epochs', type=int, default=500,help='Number of epoches training. Default: 500')
parser.add_argument('--batch_size', type=int, default=200,help='Number of batch size when training. Default: 200')
parser.add_argument('--bottleneck', type=int, default=32,help='Size of the bottleneck layer of the model. Default: 32')
parser.add_argument('--dimreduce', type=str, default="AE",help='Encoder model type. Can be AE or VAE. Default: AE')
parser.add_argument('--freeze_pretrain', type=int, default=0,help='Fix the prarmeters in the pretrained model. 0: do not freeze, 1: freeze. Default: 0')
parser.add_argument('--encoder_h_dims', type=str, default="512,256",help='Shape of the encoder. Each number represent the number of neuron in a layer. \
                    Layers are seperated by a comma. Default: 512,256')
parser.add_argument('--predictor_h_dims', type=str, default="16,8",help='Shape of the predictor. Each number represent the number of neuron in a layer. \
                    Layers are seperated by a comma. Default: 16,8')
parser.add_argument('--VAErepram', type=int, default=1)
parser.add_argument('--data_name', type=str, default="GSE110894",help='Accession id for testing data, only support pre-built data.')
# misc
parser.add_argument('--bulk_model', '-p',  type=str, default='saved/bulk_pre/',help='Path of the trained prediction model in the bulk level')
parser.add_argument('--log', '-l',  type=str, default='saved/logs/log',help='Path of training log')
parser.add_argument('--load_source_model',  type=int, default=0,help='Load a trained bulk level or not. 0: do not load, 1: load. Default: 0')
parser.add_argument('--mod', type=str, default='new',help='heterogeneous')
parser.add_argument('--printgene', type=str, default='T',help='wether print critical gene')
parser.add_argument('--dropout', type=float, default=0.3,help='dropout')

#
args, unknown = parser.parse_known_args()

In [24]:
## Testing the args covering
selected_model = files[SELECTED_FILES]
split_name = selected_model.split("_")
print(split_name[1::2])
print(split_name[0::2])

['pre\\1214data', 'drug', 'bottle', 'edim', 'pdim', 'model', 'dropout', 'gene', 'lr', 'mod', 'sam']
['F://ws//pyws//scDEAL//saved//bulk', 'GSE117872HN137', 'CISPLATIN', '64', '256,128', '128,64', 'DAE', '0.3', 'T', '0.1', 'new', 'upsampling']


In [25]:
para_names = (split_name[1::2])
paras = (split_name[0::2])

In [26]:
paras

['F://ws//pyws//scDEAL//saved//bulk',
 'GSE117872HN137',
 'CISPLATIN',
 '64',
 '256,128',
 '128,64',
 'DAE',
 '0.3',
 'T',
 '0.1',
 'new',
 'upsampling']

In [27]:
para_names

['pre\\1214data',
 'drug',
 'bottle',
 'edim',
 'pdim',
 'model',
 'dropout',
 'gene',
 'lr',
 'mod',
 'sam']

In [28]:
args.encoder_h_dims = paras[4]
args.predictor_h_dims = paras[5]
args.bottleneck = int(paras[3])
args.drug = paras[2]
args.dropout = float(paras[7])

if(paras[0].find("GSE117872")>=0):
    args.target_data = "GSE117872"
    args.batch_id = paras[0].split("GSE117872")[1]
elif(paras[0].find("MIX-Seq")>=0):
    args.target_data = "MIX-Seq"
    args.batch_id = paras[0].split("MIX-Seq")[1]    
else:
    args.target_data = paras[0]
    
args.target_model_path = selected_model

In [29]:
# Read parameters
# Extract parameters
epochs = args.epochs
dim_au_out = args.bottleneck #8, 16, 32, 64, 128, 256,512
select_drug = args.drug
na = args.missing_value
data_path = args.data
label_path = args.label
test_size = args.test_size
valid_size = args.valid_size
g_disperson = args.var_genes_disp
log_path = args.log
batch_size = args.batch_size
encoder_hdims = args.encoder_h_dims.split(",")
preditor_hdims = args.predictor_h_dims.split(",")
reduce_model = args.dimreduce
sampling = args.sampling
PCA_dim = args.PCA_dim

encoder_hdims = list(map(int, encoder_hdims) )
preditor_hdims = list(map(int, preditor_hdims) )
load_model = bool(args.load_source_model)

In [30]:

para = "1214data_"+args.data_name+"_drug_"+args.drug+"_bottle_"+str(args.bottleneck)+"_edim_"+args.encoder_h_dims+"_pdim_"+args.predictor_h_dims+"_model_"+reduce_model+"_dropout_"+str(args.dropout)+"_gene_"+args.printgene+"_lr_"+str(args.lr)+"_mod_"+args.mod+"_sam_"+str(args.sampling)
#(para)
now=time.strftime("%Y-%m-%d-%H-%M-%S")

#print(preditor_path )
#model_path = args.bulk_model + para 
preditor_path = args.bulk_model + para 
bulk_encoder = args.bulk_encoder+para
# Read data
data_r=pd.read_csv(data_path,index_col=0)
label_r=pd.read_csv(label_path,index_col=0)
#label_r=label_r.fillna(na)
ut.save_arguments(args,now)


# Initialize logging and std out
out_path = log_path+now+".err"
log_path = log_path+now+".log"

out=open(out_path,"w")
sys.stderr=out


logging.basicConfig(level=logging.INFO,
                filename=log_path,
                filemode='a',
                format=
                '%(asctime)s - %(pathname)s[line:%(lineno)d] - %(levelname)s: %(message)s'
                )
logging.getLogger('matplotlib.font_manager').disabled = True

#logging.info(args)

In [31]:
# Filter out na values
selected_idx = label_r.loc[:,select_drug].dropna()
if(g_disperson!=None):
    hvg,adata = ut.highly_variable_genes(data_r,min_disp=g_disperson)
    # Rename columns if duplication exist
    data_r.columns = adata.var_names
    # Extract hvgs
    data = data_r.loc[selected_idx.index,hvg]
else:
    data = data_r.loc[selected_idx.index,:]

# Do PCA if PCA_dim!=0
if PCA_dim !=0 :
    data = PCA(n_components = PCA_dim).fit_transform(data)
else:
    data = data

# Extract labels
label = label_r.loc[selected_idx.index,select_drug]
data_r = data_r.loc[selected_idx.index,:]

# Scaling data
mmscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
label = label.values.reshape(-1,1)


le = LabelEncoder()
label = le.fit_transform(label)
dim_model_out = 2

#label = label.values.reshape(-1,1)

logging.info(np.std(data))
logging.info(np.mean(data))

# Split traning valid test set
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=test_size, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=valid_size, random_state=42)
# sampling method
if sampling == "no":
    X_train,Y_train=sam.nosampling(X_train,Y_train)
    logging.info("nosampling")
elif sampling =="upsampling":
    X_train,Y_train=sam.upsampling(X_train,Y_train)
    logging.info("upsampling")
elif sampling =="downsampling":
    X_train,Y_train=sam.downsampling(X_train,Y_train)
    logging.info("downsampling")
elif  sampling=="SMOTE":
    X_train,Y_train=sam.SMOTEsampling(X_train,Y_train)
    logging.info("SMOTE")
else:
    logging.info("not a legal sampling method")

# Select the Training device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
#print(device)
# Assuming that we are on a CUDA machine, this should print a CUDA device:
#logging.info(device)
torch.cuda.set_device(device)
print(device)
# Construct datasets and data loaders
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)

Y_trainTensor = torch.LongTensor(Y_train).to(device)
Y_validTensor = torch.LongTensor(Y_valid).to(device)

# Preprocess data to tensor
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)

# construct TensorDataset
trainreducedDataset = TensorDataset(X_trainTensor, Y_trainTensor)
validreducedDataset = TensorDataset(X_validTensor, Y_validTensor)

trainDataLoader_p = DataLoader(dataset=trainreducedDataset, batch_size=batch_size, shuffle=True)
validDataLoader_p = DataLoader(dataset=validreducedDataset, batch_size=batch_size, shuffle=True)
bulk_X_allTensor = torch.FloatTensor(data).to(device)
bulk_Y_allTensor = torch.LongTensor(label).to(device)
dataloaders_train = {'train':trainDataLoader_p,'val':validDataLoader_p}
print("bulk_X_allRensor",bulk_X_allTensor.shape)


cuda:0
bulk_X_allRensor torch.Size([1208, 15962])


In [32]:

# Defined the model of predictor 
if reduce_model == "AE":
    model = PretrainedPredictor(input_dim=X_train.shape[1],latent_dim=dim_au_out,h_dims=encoder_hdims, 
                            hidden_dims_predictor=preditor_hdims,output_dim=dim_model_out,
                            pretrained_weights=None,freezed=bool(args.freeze_pretrain),drop_out=args.dropout,drop_out_predictor=args.dropout)
if reduce_model == "DAE":
    model = PretrainedPredictor(input_dim=X_train.shape[1],latent_dim=dim_au_out,h_dims=encoder_hdims, 
                            hidden_dims_predictor=preditor_hdims,output_dim=dim_model_out,
                            pretrained_weights=None,freezed=bool(args.freeze_pretrain),drop_out=args.dropout,drop_out_predictor=args.dropout)                                
elif reduce_model == "VAE":
    model = PretrainedVAEPredictor(input_dim=X_train.shape[1],latent_dim=dim_au_out,h_dims=encoder_hdims, 
                    hidden_dims_predictor=preditor_hdims,output_dim=dim_model_out,
                    pretrained_weights=None,freezed=bool(args.freeze_pretrain),z_reparam=bool(args.VAErepram),drop_out=args.dropout,drop_out_predictor=args.dropout)
#print("@@@@@@@@@@@")
logging.info("Current model is:")
logging.info(model)
#if torch.cuda.is_available():
#    model.cuda()
model.to(device)



PretrainedPredictor(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=15962, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=128, out_features=64, bias=True)
  (predictor): Predictor(
    (predictor): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.3, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-

In [33]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)


loss_function = nn.CrossEntropyLoss()

exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)


In [34]:
model.load_state_dict(torch.load(selected_model)) 

<All keys matched successfully>

In [35]:
from sklearn.dummy import DummyClassifier

dl_result = model(X_testTensor).detach().cpu().numpy()

lb_results = np.argmax(dl_result,axis=1)
#pb_results = np.max(dl_result,axis=1)
pb_results = dl_result[:,1]

report_dict = classification_report(Y_test, lb_results, output_dict=True)
report_df = pd.DataFrame(report_dict).T
ap_score = average_precision_score(Y_test, pb_results)
auroc_score = roc_auc_score(Y_test, pb_results)

report_df['auroc_score'] = auroc_score
report_df['ap_score'] = ap_score

report_df.to_csv("saved/logs/" + reduce_model + select_drug+now + '_report.csv')

#logging.info(classification_report(Y_test, lb_results))
#logging.info(average_precision_score(Y_test, pb_results))
#logging.info(roc_auc_score(Y_test, pb_results))

model = DummyClassifier(strategy='stratified')
model.fit(X_train, Y_train)
yhat = model.predict_proba(X_test)
naive_probs = yhat[:, 1]

ut.plot_roc_curve(Y_test, naive_probs, pb_results, title=str(roc_auc_score(Y_test, pb_results)),
                    path="saved/figures/" + reduce_model + select_drug+now + '_roc.pdf')
ut.plot_pr_curve(Y_test,pb_results,  title=average_precision_score(Y_test, pb_results),
                path="saved/figures/" + reduce_model + select_drug+now + '_prc.pdf')
print("bulk_model finished")


bulk_model finished
